<a href="https://colab.research.google.com/github/Bc-AJPR/Amazon-s_big_data/blob/main/Level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [8]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-29 22:26:05--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-05-29 22:26:05 (5.92 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [11]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...|2015-01-31 00:08:00|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...| 

In [12]:
# Count the number of the reviews 
print(df.count())

1741100


In [13]:
# Drop duplicates and incomplete rows 
dropped_df = df.dropna()
print(dropped_df.count())

clean_df = dropped_df.dropDuplicates()
print(clean_df.count())

1740974
1740974


In [16]:
# Print schema
clean_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [17]:

# Changing datatypes
# Source Link: https://stackoverflow.com/questions/32284620/how-to-change-a-dataframe-column-from-string-type-to-double-type-in-pyspark

from pyspark.sql.types import TimestampType, IntegerType

clean_df = clean_df.withColumn("customer_id", clean_df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", clean_df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", clean_df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", clean_df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", clean_df["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", clean_df["total_votes"].cast(IntegerType()))\

# Print schema
clean_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [18]:
# Create review dataframe to match review_id_table table
review_id_df = clean_df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R100T355V99IID|   40190651|B0030IRKHK|     146987318|2015-01-11 00:03:00|
|R100TPQ3931RDP|    8632815|B007S8M0FA|     944178209|2014-01-04 00:11:00|
|R1010PXKUABC7W|   45427875|B000EDVU16|     417114915|2014-01-28 00:10:00|
|R1019SWPVIE9HU|   13666734|B001BNO7AE|     401304039|2015-01-08 00:07:00|
|R101ET84WIOZ5Q|   18453943|B000ST3K3W|     659433120|2014-01-01 00:09:00|
|R101FPWBD8AJ71|   36534566|B0012RWCF6|       4853427|2010-01-10 00:11:00|
|R102PBCCJ27G4G|   51132788|B001EX35U4|     712111499|2013-01-03 00:03:00|
|R103PN3YHPJMT6|   37839622|B0013E45PS|     271576246|2014-01-05 00:12:00|
|R104D8HRTE2Y9M|   48818949|B001CBVM12|      17049624|2015-01-15 00:08:00|
|R104HRVWBFR9XB|   27403242|B0002198GY|     893679678|2015-01-12 00:01:00|
|R104VJT5ZIT9FZ|   454993

In [19]:
# Print schema
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [21]:
# Create customer dataframe to match customers table
from pyspark.sql.functions import desc

customers_df = clean_df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50364105|           301|
|   51956455|           249|
|   24268508|           249|
|   50343989|           191|
|   12980136|           183|
|   25355517|           180|
|   10554860|           165|
|   12904324|           160|
|   38221826|           159|
|   19123471|           158|
|   33492248|           155|
|   33192527|           154|
|   14397465|           152|
|   50820654|           150|
|   50669849|           148|
|   38299563|           143|
|   30260350|           143|
|   22478751|           141|
|   25156002|           136|
|   53090971|           136|
+-----------+--------------+
only showing top 20 rows



In [22]:
# Create product dataframe to match products table(only unique values)
products_df = clean_df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00002X1ZX|Stanley Tools 28-...|
|B00A4YH55A|ThruNite TN32 UT/...|
|B009XMFYKC|Loggerhead BG8-01...|
|B00T96T070|UltraFire WF-68B ...|
|B0002BBYIE|SE 813PK 16-Piece...|
|B0037M4R7A|Nagura Stone, 7/8...|
|B000BZ4T10|Ironclad High Dex...|
|B00EQ3OCNI|Fotga DP3000 M3 F...|
|B00006I5F8|Leatherman Univer...|
|B00006S7D0|Black & Decker XD...|
|B0068AV4R4|POWERTEC 110280 6...|
|B0000224VA|Estwing 16 oz Str...|
|B008UQ0F9M|Gladiator Full Do...|
|B0045W0MD4|The American Red ...|
|B000P4LLH2|CMT 537.350.31 Fo...|
|B0068V1BIK|3/8 inch Drive Cl...|
|B0039D02S0|Streamlight 88863...|
|B00012Y38W|Bondhus 20199 Bal...|
|B000JNNWQ2|IRWIN Tools VISE-...|
|B00002N7RE|Crescent Adjustab...|
+----------+--------------------+
only showing top 20 rows



In [23]:
# Create vine dataframe to match vine_table table

# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = clean_df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# # Filter for only columns from Amazon's Vine program
# vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100T355V99IID|          5|            0|          0|   N|
|R100TPQ3931RDP|          1|            0|          0|   N|
|R1010PXKUABC7W|          5|            0|          1|   N|
|R1019SWPVIE9HU|          5|            0|          0|   N|
|R101ET84WIOZ5Q|          4|            0|          0|   N|
|R101FPWBD8AJ71|          4|            1|          2|   N|
|R102PBCCJ27G4G|          1|            1|          2|   N|
|R103PN3YHPJMT6|          4|            0|          1|   N|
|R104D8HRTE2Y9M|          5|            0|          0|   N|
|R104HRVWBFR9XB|          5|            0|          0|   N|
|R104VJT5ZIT9FZ|          5|            0|          0|   N|
|R1050IM0HAVKHI|          5|            3|          4|   N|
|R105JVVCLH4MBG|          4|            0|          0|   N|
|R105O2T37XKXQA|          4|            

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/outdoors_db"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}